In [1]:
import numpy as np

import objective_function as obj

In [10]:
x = np.array([[4, 4, 2],
              [8, 9, 4],
              [20, 20, 10],
              [1, 1, 1],
              [2, 2, 2],
              [1, 1, 1],
              [1, 1, 1],
              [2, 2, 2],
              [2, 2, 2]])

obj.calculate_objective(x=x*10)

(inf, True, False)

In [ ]:
def genetic_algorithm():
    pass

### Chromosomes
- Value encoding --> Each gene represents xij
- Must be a non-negative integer

### Natural selection: 
1. Proportional selection --> Chance proportional to fitness function
2. Weighted relative fitness --> Chance proportional to the difference over the worst fitness
3. Rank-based selection --> Near termination, chance proportional to rank
4. Tournament selection --> Randomly select k1 parents then select k2 best parents among them

### Crossover
1. Multi-point crossover --> Switch whole sections
2. Uniform crossover --> Choose each gene from a random parent
3. Arithmetic mean crossover --> Mean of 2 numbers

### Mutation
1. Uniform mutation --> Change a single value up or down
2. Swap/Scramble mutation --> Swap 2 values